# 與 07 差別在於 07 是 embedding + Linear, 而 08 則是 embedding + RNN

# 文本分類 - RNN

## 判斷 GPU 是否存在

In [11]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## 載入 AG News 資料集

In [12]:
import torch
from torchtext.datasets import AG_NEWS

news = AG_NEWS(split = "train")
type(news)

torch.utils.data.datapipes.iter.grouping.ShardingFilterIterDataPipe

- 訓練數據迭代器

In [13]:
train_iter = iter(AG_NEWS(split = "train"))

- 取得下一筆資料

In [14]:
next(train_iter)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

- 再取下一筆

In [15]:
next(train_iter)

(3,
 'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.')

## 詞彙表處理

In [16]:
# 導入分詞工具
from torchtext.data.utils import get_tokenizer
# 使用迭代器建構詞彙表
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer("basic_english")

# 建立 Generator (分詞生成器)
def yield_tokens(data_iter):
    for _, text in data_iter:    # 獲取每一條的標籤 label 和內容 text
        yield tokenizer(text)   # 獲取內容分詞, 並返回. yield 返回一個迭代器對象

# 由 train_iter 建立詞彙字典
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials = ["<unk>"])

# 設定預設的索引值
vocab.set_default_index(vocab["<unk>"])

In [17]:
# 測試詞彙字典, 取得單字的索引值
vocab(["here", "is", "an", "example"])

[475, 21, 30, 5297]

## 設定超參數

In [18]:
EPOCHS = 10
LR = 5
BATCH_SIZE = 64

# 取得標註個數, 詞彙表大小, 詞嵌入維度

In [19]:
num_class = len(set([label for (label, text) in news]))

vocab_size = len(vocab)

emsize = 64

## 定義資料轉換函數

In [20]:
# 分詞, 取得單字的索引值
# 獲取每一條的 text 的索引表示
text_pipeline = lambda x: vocab(tokenizer(x))

# 換成索引值, 獲取對應的 label
label_pipeline = lambda x: int(x) - 1

## 建立 DataLoader

In [22]:
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

# 批次處理
# 定義 collate_batch 函數, 在 DataLoader 中會使用, 對傳入的樣本數據進行批量處理
def collate_batch(batch):
    # 用 list 存放 label, text 及 offsets
    # label_list: batch 中, 每個 text 的 label
    # text_list: batch 中, 每個 text 轉換成詞彙表的索引
    # 其中 offsets 是用來存放每條 text 的偏移量 (batch 中, 每個 text 的長度)
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype = torch.int64)
        text_list.append(processed_text)
        # 將每一條數據的長度放入 offsets 列表中
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype = torch.int64)
    # 計算出每一條 text 的偏移量, 其中 cumsum 用來計算一個數組各行的累加值
    offsets = torch.tensor(offsets[:-1]).cumsum(dim = 0)    # 單字的索引值累加
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_iter, test_iter = AG_NEWS()
# 轉換為 Dataset
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

# 資料切割, 95% 作為訓練資料
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

# 建立 Dataloader
train_dataloader = DataLoader(split_train_, batch_size = BATCH_SIZE,
                                shuffle = True, collate_fn = collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size = BATCH_SIZE,
                                shuffle = True, collate_fn = collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size = BATCH_SIZE,
                                shuffle = True, collate_fn = collate_batch)

## 建立模型 (RNN model)

In [23]:
from torch import nn

class TextClassficationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, num_class, sparse = True)
        self.rnn = nn.RNN(embed_dim, 32)
        self.fc = nn.Linear(32, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        rnn_out, h_out = self.rnn(embedded)
        return self.fc(rnn_out)

model = TextClassficationModel(vocab_size, emsize, num_class).to(device)

## 定義損失函數, 優化器及學習率調整器

In [25]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma = 0.1)

## 定義訓練及評估函數

In [26]:
import time

# 訓練函數
def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print("| epoch {:3d} | {:5d}/{:5d} batches "
                    "| accuracy {:8.3f}".format(epoch, idx, len(dataloader), 
                                                total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

# 評估函數
def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

## 模型訓練

In [27]:
total_accu = None
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print("-" * 59)
    print("| end of epoch {:3d} | time: {:5.2f}s | "
            "valid acuracy {:8.3f} ".format(epoch,
                                            time.time() - epoch_start_time,
                                            accu_val))
    print("-" * 59)

| epoch   1 |   500/ 1782 batches | accuracy    0.558
| epoch   1 |  1000/ 1782 batches | accuracy    0.766
| epoch   1 |  1500/ 1782 batches | accuracy    0.820
-----------------------------------------------------------
| end of epoch   1 | time:  8.93s | valid acuracy    0.843 
-----------------------------------------------------------
| epoch   2 |   500/ 1782 batches | accuracy    0.857
| epoch   2 |  1000/ 1782 batches | accuracy    0.866
| epoch   2 |  1500/ 1782 batches | accuracy    0.872
-----------------------------------------------------------
| end of epoch   2 | time:  8.28s | valid acuracy    0.867 
-----------------------------------------------------------
| epoch   3 |   500/ 1782 batches | accuracy    0.891
| epoch   3 |  1000/ 1782 batches | accuracy    0.889
| epoch   3 |  1500/ 1782 batches | accuracy    0.890
-----------------------------------------------------------
| end of epoch   3 | time:  8.36s | valid acuracy    0.882 
----------------------------------

## 模型評估

In [28]:
print(f"測試資料準確度: {evaluate(test_dataloader)}")

測試資料準確度: 0.9072368421052631


## 測試新資料

In [29]:
# 新聞類別
ag_news_label = {1: "World",
                2: "Sports",
                3: "Business",
                4: "Sci/Tec"}

# 預測
def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

# 測試資料
ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

model = model.to("cpu")
res = predict(ex_text_str, text_pipeline)
print("This is a %s news" %ag_news_label[res])

This is a Sports news


In [33]:
my_test = open("./nlp_data/news.txt", encoding = "utf8").read()
print(ag_news_label[predict(my_test, text_pipeline)])

Sports
